# Book Recommender System

In [1]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
from dgl.nn import GraphConv
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv("Data/Ratings.csv")
#print(f"{ratings.dtypes}\n")
#print(ratings.head())
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [3]:
ratings["Book-Rating"].value_counts()

0     716109
8     103736
10     78610
7      76457
9      67541
5      50974
6      36924
4       8904
3       5996
2       2759
1       1770
Name: Book-Rating, dtype: int64

In [4]:
books = pd.read_csv("Data/Books.csv", dtype={3: str})
#print(f"Books Data Shape: {books.shape} \n")
#print(f"{books.dtypes}\n")
#print(books.head(3))

books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [5]:
print('Nan-values by column')
print(books.isna().sum())

Nan-values by column
ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [6]:
users = pd.read_csv("Data/Users.csv")
#print(f"Users Data Shape: {users.shape} \n")
#print(f"{users.dtypes}\n")
#print(users.head())
#print('Nan-values by column')
#print(users.isna().sum())

users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


## Preprocessing

In [7]:
# We will only use users and books present in the ratings dataset 
lessen_user_ids = {userid: idx for idx, userid in enumerate(ratings['User-ID'].unique())} #renumber IDs to reduce inactive users
ratings['New-User-ID'] = ratings['User-ID'].map(lessen_user_ids)
user_ids = list(ratings['New-User-ID'].unique())
num_users = len(set(user_ids))

# Map book identifiers (ISBN) to a unique integer identifier for datatype compatibility of dgl
isbn_to_id = {isbn: idx for idx, isbn in enumerate(ratings['ISBN'].unique())}
ratings['Book-ID'] = ratings['ISBN'].map(isbn_to_id)
book_ids = list(ratings['Book-ID'].unique())
num_books = len(set(book_ids))

print(f'There are {len(user_ids)} unique users, and {len(book_ids)} unique books in the ratings dataset.')
 
# Remove users and books not included in the ratings dataset
books['Book-ID'] = books['ISBN'].map(isbn_to_id)
books_clean = books[books['Book-ID'].isin(book_ids)]
books_clean_ids = books_clean['Book-ID'].unique()
percent_books_missing = round((num_books-len(books_clean_ids))/num_books*100, 0)

print(f'There are around {percent_books_missing}% of books in the graph missing in the books data')

users['New-User-ID'] = users['User-ID'].map(lessen_user_ids)
users_clean = users[users['New-User-ID'].isin(user_ids)]

There are 105283 unique users, and 340556 unique books in the ratings dataset.
There are around 21.0% of books in the graph missing in the books data


1/5 of the books that have rating information do not have further information on the books dataset. However, as our objective is to investigate a user-based recommender system, this is irrelevant. We are able to embed the age and location data of users. As the age data is sparse, location data will be our main source of information.

## Collaborative Filtering

The following is based on [this Kaggle documentation](https://www.kaggle.com/code/gspmoreira/recommender-systems-in-python-101)

"This method makes automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on a set of items, A is more likely to have B's opinion for a given item than that of a randomly chosen person."

In [38]:
ratings_with_book_titles = ratings.merge(books,on='ISBN')
ratings_with_book_titles.drop(columns=["ISBN","Image-URL-S","Image-URL-M"],axis=1,inplace=True)
complete_df = ratings_with_book_titles.merge(users.drop("Age", axis=1), on="User-ID")

In [37]:
complete_df['Location'] = complete_df['Location'].str.split(',').str[-1].str.strip()
complete_df.head()

,User-ID,Book-Rating,New-User-ID_x,Book-ID_x,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Book-ID_y,Location,New-User-ID_y
0,276725,0,0,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,0.0,usa,0.0
1,2313,5,1655,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,0.0,usa,1655.0
2,2313,9,1655,383,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,http://images.amazon.com/images/P/0812533550.0...,383.0,usa,1655.0
3,2313,8,1655,443,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,http://images.amazon.com/images/P/0679745580.0...,443.0,usa,1655.0
4,2313,9,1655,1313,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,http://images.amazon.com/images/P/0060173289.0...,1313.0,usa,1655.0


In [39]:
# Select user IDs with more than 200 book ratings
min_ratings_threshold = 200

# Count book ratings per user
num_ratings_per_user = complete_df.groupby('User-ID')['Book-Rating'].count()

# Filter users with more than the minimum threshold
knowledgeable_user_ids = num_ratings_per_user[num_ratings_per_user > min_ratings_threshold].index

In [40]:
# Filter ratings from knowledgeable users
knowledgeable_user_ratings = complete_df[complete_df['User-ID'].isin(knowledgeable_user_ids)]

In [41]:
min_ratings_count_threshold=50
rating_counts= knowledgeable_user_ratings.groupby('Book-Title').count()['Book-Rating']
popular_books = rating_counts[rating_counts >= min_ratings_count_threshold].index

In [42]:
final_ratings =  knowledgeable_user_ratings[knowledgeable_user_ratings['Book-Title'].isin(popular_books)]

In [44]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID'
                          ,values='Book-Rating')
pt.fillna(0,inplace=True)
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
from sklearn.metrics.pairwise import cosine_similarity 

In [50]:
similarity_score = cosine_similarity(pt)

In [51]:
def recommend(book_name):
    index = np.where(pt.index==book_name)[0][0]
    similar_books = sorted(list(enumerate(similarity_score[index])),key=lambda x:x[1], reverse=True)[1:6]
    
    data = []
    
    for i in similar_books:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
 
        data.append(item)
    return data

In [52]:
recommend("A Walk to Remember")

[['The Rescue', 'Nicholas Sparks'],
 ['Nights in Rodanthe', 'Nicholas Sparks'],
 ['The Notebook', 'Nicholas Sparks'],
 ['Granny Dan', 'DANIELLE STEEL'],
 ['A Bend in the Road', 'Nicholas Sparks']]

In [53]:
# Install Surprise library
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp39-cp39-macosx_10_9_x86_64.whl size=511115 sha256=7b22a61ff430dea8a42530e0b6ac26a2d479704d89f7c20c638bb6ca973bf056
  Stored in directory: /Users/a788/Library/Caches/pip/wheels/42/41/d3/a56ae864ad22cc6583ec9312be43fbc611c87e53dc49aac953
Successfully built scikit-surprise


In [63]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Define the rating scale
reader = Reader(rating_scale=(0, 10))

# Load the data into Surprise's dataset format
data = Dataset.load_from_df(complete_df[['User-ID', 'Book-Title', 'Book-Rating']], reader)

# Split the dataset into training and testing sets
train_set, test_set = train_test_split(data, test_size=0.20, random_state=42)

# Define the SVD algorithm
model = SVD()

# Train the algorithm on the training set
model.fit(train_set)

# Make predictions on the test set
predictions = model.test(test_set)

## Evaluations

In [64]:
# Evaluate the model
accuracy.rmse(predictions)

RMSE: 3.5175


3.517453882848987

In [61]:
from collections import defaultdict
from surprise import accuracy

def precision_recall_at_k(predictions, k=10, threshold=4.0):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

def calculate_f1(precisions, recalls):
    f1_scores = {}
    for uid in precisions:
        if precisions[uid] + recalls[uid] > 0:
            f1_scores[uid] = 2 * (precisions[uid] * recalls[uid]) / (precisions[uid] + recalls[uid])
        else:
            f1_scores[uid] = 0
    return f1_scores

precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4.0)
f1_scores = calculate_f1(precisions, recalls)

avg_precision = sum(precisions.values()) / len(precisions)
avg_recall = sum(recalls.values()) / len(recalls)
avg_f1 = sum(f1_scores.values()) / len(f1_scores)

print(f'Precision@10: {avg_precision:.4f}')
print(f'Recall@10: {avg_recall:.4f}')
print(f'F1 Score@10: {avg_f1:.4f}')

Precision@10: 0.8624
Recall@10: 0.5069
F1 Score@10: 0.4540


In [62]:
def mean_average_precision(predictions, k=10):
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    
    average_precisions = []

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel_and_rec_k = sum(1 for (est, true_r) in user_ratings[:k] if true_r >= 4.0)
        precisions_at_k = [sum(1 for (est, true_r) in user_ratings[:i+1] if true_r >= 4.0) / (i + 1) for i in range(k)]
        average_precision = sum(precisions_at_k[:n_rel_and_rec_k]) / min(k, n_rel_and_rec_k) if n_rel_and_rec_k != 0 else 0
        average_precisions.append(average_precision)

    return sum(average_precisions) / len(average_precisions)

# Example usage
map_score = mean_average_precision(predictions, k=10)
print(f'MAP@10: {map_score:.4f}')

MAP@10: 0.5446


# Setting up data structures for GNN

In [ ]:
# Make graph
src = torch.tensor(ratings['New-User-ID'].values)
dst = torch.tensor(ratings['Book-ID'].values)

edges = {
    ('user', 'rating', 'book'): (src, dst)
}

g = dgl.heterograph(edges, num_nodes_dict={'user': num_users, 'book': num_books})
print(g)

In [ ]:
# Weight the edges by ratings
rating_data = ratings['Book-Rating'].values
ratings_tensor = torch.tensor(rating_data, dtype=torch.float32)
g.edges['rating'].data['rating'] = ratings_tensor 

In [ ]:
# Add age to user feature
ages = users_clean['Age'].values
ages_tensor = torch.tensor(ages, dtype=torch.float32)
g.nodes['user'].data['age'] = ages_tensor

We extract the country from the location by obtaining the expression after the last comma in e.g. nyc, new york, usa

In [ ]:
users_clean['Country'] = users_clean['Location'].str.rsplit(',', n=1).str[-1].str.strip()
country_counts = users_clean['Country'].value_counts(normalize=True) 

# We see that less frequent locations do not always contain country names, so we remove values of locations representing less than 1%
rare_countries = country_counts[country_counts < 0.01].index
users_clean.loc[users_clean['Country'].isin(rare_countries), 'Country'] = np.nan

country_ids = {country: idx for idx, country in enumerate(users_clean['Country'].unique())}  # map country to a unique integer
users_clean['CountryId'] = users_clean['Country'].map(country_ids)
countries = users_clean['CountryId'].values
countries_tensor = torch.tensor(countries, dtype=torch.float32)
g.nodes['user'].data['country'] = countries_tensor

In [ ]:
print(g.nodes['user'])
print(g.nodes['book'])

### Basic graph info

In [ ]:
print(g)  # Prints the basic info of the graph, such as number of nodes and edges per type

# Print number of nodes for each type
print("Number of users:", g.number_of_nodes('user'))
print("Number of books:", g.number_of_nodes('book'))

# Print number of edges
print("Number of ratings:", g.number_of_edges('rating'))

### Node and Edge feature inspection

In [ ]:
# Print user node features
print("User features:", g.nodes['user'].data.keys())

# Print book node features, if any
print("Book features:", g.nodes['book'].data.keys())

# Print edge features
print("Edge features:", g.edges['rating'].data.keys())

# Example to print specific feature details:
print("Sample user ages:", g.nodes['user'].data['age'][:5])  # prints first 5 user ages
print("Sample ratings:", g.edges['rating'].data['rating'][:5])  # prints first 5 ratings

### Eliminate isolated nodes if any

In [ ]:
compact_g = dgl.compact_graphs(g)

### Create synthetic features for book based on degree of the node

In [ ]:
book_in_degrees = g.in_degrees(etype=('user', 'rating', 'book')).float().unsqueeze(1)

In [ ]:
g.nodes['book'].data['in_degree'] = book_in_degrees

In [ ]:
data = g
data

In [ ]:
node_types = g.ntypes
edge_types = g.etypes
print("Node types:", node_types)
print("Edge types:", edge_types)
print('Number of rating edges:', g.number_of_edges('rating'))

# Creating the GNN

## Architecture

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import SAGEConv

class GNNEncoder(nn.Module):
    def __init__(self, in_feats, hidden_channels):
        super(GNNEncoder, self).__init__()
        self.conv1 = SAGEConv(in_feats, hidden_channels, 'mean')
        self.conv2 = SAGEConv(hidden_channels, hidden_channels, 'mean')

    def forward(self, g, features):
        h = self.conv1(g, features)
        h = F.relu(h)
        h = self.conv2(g, h)
        print(f"Encoder output shape: {h.shape}")  # Debugging output
        return h

class BookEmbedding(nn.Module):
    def __init__(self, num_books, embedding_dim):
        super(BookEmbedding, self).__init__()
        self.book_embedding = nn.Embedding(num_books, embedding_dim)

    def forward(self, book_ids):
        embeddings = self.book_embedding(book_ids)
        print(f"Book embedding shape: {embeddings.shape}") 
        return embeddings

class EdgeDecoder(nn.Module):
    def __init__(self, hidden_channels):
        super(EdgeDecoder, self).__init__()
        self.lin1 = torch.nn.Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = torch.nn.Linear(hidden_channels, 1)

    def forward(self, g, h_user, h_book):
        with g.local_scope():
            g.nodes['user'].data['h'] = h_user
            g.nodes['book'].data['h'] = h_book
            
            # Ensure edge function is only applied where it should be
            if 'rating' in g.etypes:
                g.apply_edges(func=lambda edges: {
                    'score': self.lin2(F.relu(self.lin1(torch.cat([edges.src['h'], edges.dst['h']], dim=1))))
                }, etype='rating')
            return g.edata.get('score', torch.tensor([]))  # Handle case where no edges exist


class Model(nn.Module):
    def __init__(self, num_books, in_feats_user, hidden_channels):
        super(Model, self).__init__()
        self.user_encoder = GNNEncoder(in_feats_user, hidden_channels)
        self.book_embedding = BookEmbedding(num_books, hidden_channels)
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, g, user_features, book_ids):
        h_user = self.user_encoder(g, user_features)
        h_book = self.book_embedding(book_ids)
        return self.decoder(g, h_user, h_book)

## Split graph for training and validation set

In [ ]:
def split_graph(g, proportion=0.8):
    # Split edges randomly for training and validation
    num_edges = g.number_of_edges('rating')
    all_edges = np.arange(num_edges)
    np.random.shuffle(all_edges)
    
    train_size = int(num_edges * proportion)
    train_edges = all_edges[:train_size]
    val_edges = all_edges[train_size:]
    
    # Create subgraphs based on the edges
    g_train = dgl.edge_subgraph(g, train_edges, relabel_nodes=False)
    g_val = dgl.edge_subgraph(g, val_edges, relabel_nodes=False)
    
    return g_train, g_val

In [ ]:
g_train, g_val = split_graph(g, proportion=0.8)

age_tensor_val = g_val.nodes['user'].data['age'].unsqueeze(1)
country_tensor_val = g_val.nodes['user'].data['country'].unsqueeze(1)
user_features_val = torch.cat([age_tensor_val, country_tensor_val], dim=1)

# Assuming book_ids are just the indices of the books, adjust if necessary
book_ids_train = torch.arange(g_train.number_of_nodes('book'))
book_ids_val = torch.arange(g_val.number_of_nodes('book'))

## Train

`criterion` - MSE loss that calculates the average squared difference between the predicted outputs and the actual target values.

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
book_ids_train = g_train.nodes['book'].data['orig_id']
book_ids_val = g_val.nodes['book'].data['orig_id']

In [ ]:
def train(model, g, user_features, book_ids, ratings, optimizer, criterion):
    model.train()  # Set the model to training mode
    optimizer.zero_grad()  # Clear gradients before each backward pass

    # Move data to the correct device
    g = g.to(device)
    user_features = user_features.to(device)
    book_ids = book_ids.to(device)
    ratings = ratings.to(device)

    # Forward pass
    predictions = model(g, user_features, book_ids)
    loss = criterion(predictions, ratings)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    return loss.item()

In [ ]:
def validate(model, g, user_features, book_ids, ratings, criterion):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation for evaluation, which saves memory and computations
        g = g.to(device)
        user_features = user_features.to(device)
        book_ids = book_ids.to(device)
        ratings = ratings.to(device)

        # Forward pass
        predictions = model(g, user_features, book_ids)
        loss = criterion(predictions, ratings)

    return loss.item()

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    train_loss = train(model, g_train, user_features_train, book_ids_train, ratings_train, optimizer, criterion)
    val_loss = validate(model, g_val, user_features_val, book_ids_val, ratings_val, criterion)
    print(f'Epoch {epoch + 1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

## Evaluate model

In [ ]:
def evaluate(model, g, user_features, book_features, labels, criterion):
    model.eval()
    with torch.no_grad():
        outputs = model(g, user_features, book_features)
        loss = criterion(outputs, labels)
    return loss.item()

In [ ]:
# Validation loss
validation_loss = evaluate(model, g_val, user_features_val, book_features_val, ratings_val, criterion)
print(f'Validation Loss: {validation_loss}')